# __Search Text by Text__

- Tutorial Difficulty: ★★☆☆☆
- 7 min read 
- Languages: [SQL](https://en.wikipedia.org/wiki/SQL) (100%)
- File location: tutorial_en/thanosql_search/search_text_by_text.ipynb   
- References: [(Kaggle) IMDB Movie Reviews](https://www.kaggle.com/code/lakshmi25npathi/sentiment-analysis-of-imdb-movie-reviews/data), [Word Embeddings: LEXICAL SEMANTICS Encoding](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

## Tutorial Introduction

<div class="admonition note">
    <h4 class="admonition-title">Understanding Text Vectorization</h4>
    <p>Computers cannot directly interpret human language(natural language). Therefore, a process for converting natural language into numerical data that can be recognized by computers is required. In the field of natural language processing, embedding refers to the result of converting human natural language into a vectorized format, which is a form that can be understood by machines.</p>
</div>

Techniques for converting natural language into embeddings are largely divided into statistical techniques and artificial neural network-based techniques. ThanoSQL provides a method to train a text vectorization model using self-supervised learning.

<div class="admonition note">
    <h4 class="admonition-title">In this tutorial</h4>
    <p>👉 Uses movie review data. The data consists of movie review texts and label values. However, because we are demonstrating self-supervised learning, this tutorial does not use label values. By training a model with 4,000 movie reviews, we will be able to search text by text and extract the importance of each word from a given movie review. </p>
</div>

## __0. Prepare Dataset__

As mentioned in the [ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/how_to_use_ThanoSQL/#5-thanosql-workspace), you must create an API token and run the query below to execute the query of ThanoSQL. 

In [ ]:
%load_ext thanosql
%thanosql API_TOKEN=<Issued_API_TOKEN>

### __Prepare Dataset__

In [2]:
%%thanosql
GET THANOSQL DATASET movie_review_data
OPTIONS (overwrite=True)

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>GET THANOSQL DATASET</strong>" downloads the specified dataset to the workspace. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>GET THANOSQL DATASET</strong> clause.
        <ul>
            <li>"overwrite": determines whether to overwrite a dataset if it already exists. If set as True, the old dataset is replaced with the new dataset (True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

In [3]:
%%thanosql
COPY movie_review_train
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_train.csv"

Success


In [4]:
%%thanosql
COPY movie_review_test 
OPTIONS (overwrite=True) 
FROM "thanosql-dataset/movie_review_data/movie_review_test.csv"

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>COPY</strong>" specifies the name of the dataset to be saved as a database table. </li>
        <li>"<strong>OPTIONS</strong>" specifies the option values to be used for the <strong>COPY</strong> clause.
        <ul>
           <li>"overwrite": determines whether to overwrite a table if it already exists. If set as True, the old table is replaced with the new table (True|False, default: False) </li>
        </ul>
        </li>
    </ul>
</div>

## __1. Check Dataset__

To create a movie review text search model, we use the movie_review_train table from the ThanoSQL workspace database. To check the table's contents, run the following query.

In [5]:
%%thanosql
SELECT *
FROM movie_review_train
LIMIT 5 

,review,sentiment
0,This is the kind of movie that BEGS to be show...,negative
1,Bulletproof is quite clearly a disposable film...,negative
2,A beautiful shopgirl in London is swept off he...,positive
3,"VERY dull, obvious, tedious Exorcist rip-off f...",negative
4,Do we really need any more narcissistic garbag...,negative


<div class="admonition note">
   <h4 class="admonition-title">Understanding the Data</h4>
   <ul>
      <li>review: movie review in text format</li>
      <li>sentiment: target value indicating whether the review has a positive or negative sentiment</li>
   </ul>
</div>

## __2. Create a Text Vectorization Model__

To create a text search model with the name movie_text_search_model using the movie_review_train dataset, run the following query.  
(Estimated time required for query execution: 2 min)

In [6]:
%%thanosql
BUILD MODEL movie_text_search_model
USING SBERTEn
OPTIONS (
    text_col="review",
    overwrite=True
)
AS
SELECT *
FROM movie_review_train

Success


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>BUILD MODEL</strong>" creates and trains a model named movie_text_search_model.</li>
        <li>"<strong>USING</strong>" specifies SBERTEn as the base model.</li>
        <li>"<strong>OPTIONS</strong>" specifies the option values used to create a model.
        <ul>
            <li>"text_col": a column containing movie review data in the data table. (default: "text")</li>
            <li>"epochs": number of times to train with the training dataset (int, default: 1)</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training. (int, default: 16)</li> 
            <li>"learning_rate": the learning rate of the model (float, default: 3e-5)</li> 
            <li>"train": determines whether to use a pretrained model. If specified as False, the pretrained model is used. (True|False, default: True)</li> 
            <li>"overwrite": determines whether to overwrite a model if it already exists. If set as True, the old model is replaced with the new model (True|False, default: False)</li>
        </ul>
        </li>
    </ul>
</div>

To vectorize the `movie_review_test` texts run the following "__CONVERT USING__" query. The vectorized results are stored in a user-defined column (default: convert_result) in the `movie_review_test` table.

In [7]:
%%thanosql
CONVERT USING movie_text_search_model
OPTIONS (
    text_col="review",
    table_name="movie_review_test",
    batch_size=32,
    column_name="convert_result"
    )
AS 
SELECT *
FROM movie_review_test

,review,sentiment,convert_result
0,"I read the book before seeing the movie, and t...",positive,"[0.013176263, 0.0043083974, 0.032830212, -0.00..."
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[0.022792801, -0.035188578, 0.04827319, -0.039..."
2,"Yesterday I attended the world premiere of ""De...",positive,"[-0.0034791373, -0.045164507, 0.00876418, 0.02..."
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[0.013617624, -0.0016181283, 0.05108282, -0.00..."
4,"Welcome to Oakland, where the dead come out to...",positive,"[-0.020813523, -0.09764441, 0.019978669, -0.03..."
...,...,...,...
995,Ocean's 12 starts off on annoying and gets wor...,negative,"[0.015902298, -0.017285412, 0.019053567, -0.05..."
996,I remember catching this movie on one of the S...,negative,"[0.0042268047, -0.028399602, 0.0035310786, 0.0..."
997,CyberTracker is set in Los Angeles sometime in...,negative,"[0.0022029022, -0.011456593, 0.038832013, 0.00..."
998,"There is so much that is wrong with this film,...",negative,"[-0.0042857192, 0.018499052, 0.024700137, -0.0..."


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>CONVERT USING</strong>" uses <code>movie_text_search_model</code> as an algorithm for text vectorizaion.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text vectorizaion.
        <ul>
            <li>"text_col": a column containing movie review data in the data table. (default: "text")</li>
            <li>"table_name": the table name to be stored in the ThanoSQL workspace database.</li>
            <li>"batch_size": the size of dataset bundle utilized in a single cycle of training. (int, default: 16)</li>
            <li>"column_name": defines the column name that contains the vectorized results. (default: "convert_result")</li>
        </ul>
        </li>
    </ul>
</div>

## __3. Search for Similar Texts__

This step uses the movie_text_search_model text vectorization model and test table to search for similar texts.

In [8]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_input_type="text",
        search_input="This movie was my favorite movie of all time",
        emb_col="convert_result",
        column_name="score"
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

,review,sentiment,score
0,I have loved this movie since I saw it in the ...,positive,0.718619
1,"I can't explain it, but I find this movie not ...",positive,0.685227
2,This movie is Jackie's best. I still cant get ...,positive,0.683199
3,This is a delightful film. Elizabeth Taylor do...,positive,0.677734
4,I loved this movie. It is a definite inspirati...,positive,0.677151
5,I saw this movie for the first time in 1988 wh...,positive,0.674303
6,What can I say?? This movie has it all...Roman...,positive,0.672362
7,This movie was very good because it remember w...,positive,0.667647
8,"First time I saw this great movie and Alyssa, ...",positive,0.666606
9,I really enjoyed this movie. The script is fre...,positive,0.660342


In [9]:
%%thanosql
SELECT review, sentiment, score
FROM (
    SEARCH TEXT 
    USING movie_text_search_model
    OPTIONS (
        search_input_type="text",
        search_input="The movie was unsatisfactory",
        emb_col="convert_result",
        column_name="score"
        )
    AS 
    SELECT * 
    FROM movie_review_test
    )
ORDER BY score DESC 
LIMIT 10

,review,sentiment,score
0,Just plain terrible. Nick and Michael are WAY ...,negative,0.671703
1,After having red the overwhelming reviews this...,negative,0.650080
2,Badly made. Dreadful acting and an ending that...,negative,0.649521
3,"A gave it a ""2"" instead of a ""1"" (awful) becau...",negative,0.646100
4,There was absolutely nothing in this film that...,negative,0.644267
5,"While the dog was cute, the film was not. It w...",negative,0.631313
6,"We brought this film as a joke for a friend, a...",negative,0.627338
7,"I knew this movie wasn't going to be amazing, ...",negative,0.623361
8,"If this movie would have been in English, all ...",negative,0.619658
9,"Well, I remember when the studio sacked Schrad...",negative,0.619428


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for text searching.
        <ul>
                <li>"search_input_type": defines the image|text|audio|video type to be used for the search. </li>
                <li>"search_input": defines the input to be used for the search. </li>
                <li>"emb_col": the column that contains the vectorized results.</li>
                <li>"column_name": defines the name of the column that contains the search results. (default: "search_result")</li>
        </ul>
        </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for the searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __4. Extract Keywords from Texts__

This step uses the movie_text_search_model text vectorization model and test table to extract keywords from the texts.

In [10]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS 
SELECT * 
FROM movie_review_test
LIMIT 10 OFFSET 40

,review,sentiment,convert_result,keyword
0,"Like an earlier commentor, I saw it in 1980 an...",positive,"[0.010420233, -0.06322317, -0.007824576, -0.01...","{'keyword': ['story splendid acting', 'writers..."
1,"I'm a fan of Matthew Modine, but this film--wh...",negative,"[0.0031802128, 0.003040635, 0.011754154, -0.07...","{'keyword': ['all this acting', 'watched it at..."
2,Paul Lukas played a Russian intellectual makin...,positive,"[0.021621326, -0.063545, 0.010700642, -0.05007...","{'keyword': ['stanislavsky was also', 'deadpan..."
3,"to be honest, i didn't watch all of the origin...",negative,"[0.039302867, -0.0095384605, 0.005538868, -0.0...","{'keyword': ['other vampire movie', 'the actin..."
4,Police Squad! (1982) was a funny show that end...,positive,"[0.0023636217, -0.03889813, 0.013666207, 0.000...","{'keyword': ['this funny show', 'as a comedian..."
5,I am still shuddering at the thought of EVER s...,negative,"[0.0093877595, 0.0024675094, 0.016506424, -0.0...","{'keyword': ['that this movie', 'than action f..."
6,Gregory Peck gives a brilliant performance in ...,positive,"[0.030042646, 0.013456604, -0.01949762, -0.062...","{'keyword': ['this film', 'the old testament',..."
7,I first flicked onto the LoG accidentally one ...,positive,"[0.019490883, 0.0034455713, 0.011066506, -0.00...","{'keyword': ['about the humour', 'an amazing p..."
8,I didn't know what to make of this film. I gue...,positive,"[-0.011753569, 0.016743906, 0.021558706, -0.01...","{'keyword': ['view the film', 'i really like',..."
9,A family looking for some old roadside attract...,negative,"[0.033788, -0.045150697, -0.008230298, -0.0381...","{'keyword': ['poorly made horror', 'hilarious ..."


In [11]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM (
    SEARCH KEYWORD 
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        use_stopwords=True
        )
    AS 
    SELECT * 
    FROM movie_review_test
    LIMIT 10
)

,review,sentiment,keywords,score
0,"I read the book before seeing the movie, and t...",positive,"[best adaptations, the film, talented filmaker...","[0.4932, 0.4845, 0.4369, 0.4044, 0.3607]"
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,"[9/11 hosted, television show, robert deniro, ...","[0.6606, 0.5112, 0.483, 0.3889, 0.2692]"
2,"Yesterday I attended the world premiere of ""De...",positive,"[rape scene, innocent flirtation, can she, you...","[0.4536, 0.3043, 0.272, 0.2679, 0.2329]"
3,Moonwalker is a Fantasy Music film staring Mic...,positive,"[michael jackson, movie because, 6/10 segment,...","[0.5089, 0.44, 0.3292, 0.2906, 0.1642]"
4,"Welcome to Oakland, where the dead come out to...",positive,"[as douglas, guerrilla production, rocky robin...","[0.4715, 0.4333, 0.3938, 0.384, 0.3417]"
5,Tipping the Velvet (2002) (TV) was directed by...,positive,"[actors, protagonist nan, rachael stirling, by...","[0.5116, 0.4275, 0.4111, 0.3774, 0.2165]"
6,The Stock Market Crash of 1929 and the Depress...,positive,"[james cagney, dorothy lamour, between musical...","[0.5239, 0.4425, 0.4, 0.3456, 0.2608]"
7,I want to clarify a few things. I am not famil...,negative,"[art cinema, shocking scenes, taiwan censorshi...","[0.5369, 0.4901, 0.3428, 0.3219, 0.1649]"
8,This is a nice movie with good performances by...,positive,"[spanish cinema, good performances, movie is, ...","[0.6161, 0.5032, 0.4274, 0.3368, 0.301]"
9,"Once a month, I invite a few friends over for ...",negative,"[retarded movie, standard zombie, lead role, f...","[0.5416, 0.2809, 0.2683, 0.2602, 0.203]"


In [12]:
%%thanosql 
SELECT * FROM (
    SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
        FROM (
            SEARCH KEYWORD 
            USING movie_text_search_model
            OPTIONS (
                text_col="review",
                use_stopwords=True
                )
            AS 
            SELECT * 
            FROM movie_review_test
            LIMIT 10
        )
    ) 
WHERE score > 0.5

,review,sentiment,keywords,score
0,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,9/11 hosted,0.6606
1,"""9/11,"" hosted by Robert DeNiro, presents foot...",positive,television show,0.5112
2,Moonwalker is a Fantasy Music film staring Mic...,positive,michael jackson,0.5089
3,Tipping the Velvet (2002) (TV) was directed by...,positive,actors,0.5116
4,This is a nice movie with good performances by...,positive,spanish cinema,0.6161
5,The Stock Market Crash of 1929 and the Depress...,positive,james cagney,0.5239
6,I want to clarify a few things. I am not famil...,negative,art cinema,0.5369
7,This is a nice movie with good performances by...,positive,good performances,0.5032
8,"Once a month, I invite a few friends over for ...",negative,retarded movie,0.5416


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the movie_text_search_model as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for the text vectorization.</li>
        <li>"<strong>OPTIONS</strong>" specifies the options to be used for the text vectorizaion.
            <ul>
                <li>"lang": en, ko</li>
                <li>"text_col": a column containing movie review data in the data table</li>
                <li>"ngram_range": minimum and maximum number of words for each keyword. ex) [1, 3]. In most situations, keywords are extracted according to the maximum number of words. (list[int, int], default: [1, 2])</li>
                <li>"top_n": number of keywords to be extracted, in order of highest similarity (int, default: 5)</li>
                <li>"diversity": variety of keywords to be extracted. The higher the value, the more diverse the keywords will be. 0 <= diversity <= 1 (float, default: 0.5)</li>
                <li>"use_stopwords": whether to exclude words that do not have a significant meaning (True|False, default: True)</li>
                <li>"threshold": minimum value of similarity value of keywords to be extracted. (float, default: 0.0)</li>
            </ul>
            </li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __5. Combine the Two Methods__

In [14]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT 
        USING movie_text_search_model
        OPTIONS (
            search_input_type="text",
            search_input="The greatest movie of all time",
            emb_col="convert_result",
            column_name="score"
            )
        AS 
        SELECT * 
        FROM movie_review_test
        )
    ORDER BY score DESC 
    LIMIT 10
)

,review,sentiment,score,keyword
0,This is the best Chinese movie I have ever see...,positive,0.632360,"{'keyword': ['best chinese movie', 'movie defi..."
1,A labor of love. Each frame is picture perfect...,positive,0.610732,"{'keyword': ['recommend this film', 'love each..."
2,For those out there that like to think of them...,positive,0.599602,"{'keyword': ['well paced movie', 'who love fil..."
3,The beginning of this film is a little clunky ...,positive,0.594462,"{'keyword': ['outstanding the film', 'is a thr..."
4,This is the kind of film for a snowy Sunday af...,positive,0.592423,"{'keyword': ['hours wonderful performances', '..."
5,"This film may seem dated today, but remember t...",positive,0.592171,"{'keyword': ['humor this film', '1974', 'beyon..."
6,This is a delightful film. Elizabeth Taylor do...,positive,0.589568,"{'keyword': ['recommend this film', 'it s insp..."
7,This has to be one of the best movies to come ...,positive,0.584398,"{'keyword': ['equally depressing movie', 'fant..."
8,"This movie gives golf a high mark, it was well...",positive,0.578420,"{'keyword': ['movie was well', 'the brookline ..."
9,This is without a doubt the most poorly though...,negative,0.577506,"{'keyword': ['give this movie', 'most awful at..."


In [15]:
%%thanosql
SELECT review, sentiment, keyword -> 'keyword' AS keywords, keyword -> 'score' AS score
FROM ( 
    SEARCH KEYWORD
    USING movie_text_search_model
    OPTIONS (
        text_col="review",
        ngram_range=[1, 3],
        use_stopwords=True
        )
    AS (
        SELECT review, sentiment, score
        FROM (
            SEARCH TEXT 
            USING movie_text_search_model
            OPTIONS (
                search_input_type="text",
                search_input="Such a romatic movie",
                emb_col="convert_result",
                column_name="score"
                )
            AS 
            SELECT * 
            FROM movie_review_test
            )
        ORDER BY score DESC 
        LIMIT 10
    )
)

,review,sentiment,keywords,score
0,Burlinson and Thornton give an outstanding per...,positive,"[love this movie, horse scenes, beauty of natu...","[0.617, 0.5656, 0.4848, 0.4331, 0.3335]"
1,This is the kind of film for a snowy Sunday af...,positive,"[hours wonderful performances, a family film, ...","[0.6509, 0.5971, 0.3992, 0.3612, 0.2422]"
2,i would have to say that this is the first qua...,positive,"[quality romantic comedy, movie was well, stil...","[0.6547, 0.563, 0.3199, 0.255, 0.1576]"
3,A labor of love. Each frame is picture perfect...,positive,"[recommend this film, love each frame, story t...","[0.6681, 0.4809, 0.4386, 0.3238, 0.2577]"
4,There are not many movies around that have giv...,positive,"[most wonderful fantasy, bride stardust, chara...","[0.5869, 0.4047, 0.3839, 0.3661, 0.3412]"
5,"""Casomai"" is a masterful tale depicting the st...",positive,"[best italian movies, the sympathetic priest, ...","[0.6306, 0.478, 0.4677, 0.3535, 0.3465]"
6,This is a delightful film. Elizabeth Taylor do...,positive,"[recommend this film, it s inspirational, the ...","[0.7075, 0.5109, 0.3183, 0.3086, 0.2688]"
7,Forget the recent dire American remake which s...,positive,"[the intricate screenplay, french original by,...","[0.5934, 0.4793, 0.472, 0.4041, 0.3499]"
8,"This movie is light, funny, and beautifully fi...",positive,"[is absolutely superb, sentimental comedy, the...","[0.5602, 0.4795, 0.4497, 0.3787, 0.3638]"
9,"A gentle story, hinting at fury, with a redemp...",positive,"[well executed cinematographers, a film score,...","[0.6436, 0.6225, 0.5136, 0.5102, 0.3196]"


In [16]:
%%thanosql
SEARCH KEYWORD
USING movie_text_search_model
OPTIONS (
    text_col="review",
    ngram_range=[1, 3],
    use_stopwords=True
    )
AS (
    SELECT review, sentiment, score
    FROM (
        SEARCH TEXT 
        USING movie_text_search_model
        OPTIONS (
            search_input_type="text",
            search_input="The best action movie",
            emb_col="convert_result",
            column_name="score"
            )
        AS 
        SELECT * 
        FROM movie_review_test
        WHERE review LIKE '%%gun%%'
        )
    ORDER BY score DESC 
    LIMIT 10
)

,review,sentiment,score,keyword
0,As a veteran screen writing instructor at Rich...,positive,0.523923,"{'keyword': ['truly good movies', 'before holl..."
1,This is quite an unusual and unique little wes...,positive,0.507955,"{'keyword': ['western genre film', 'the movie ..."
2,What was there about 1939 that helped produce ...,positive,0.488356,"{'keyword': ['excellent hollywood films', 'pol..."
3,Countless Historical & cultural mistakes 0/10 ...,negative,0.448471,"{'keyword': ['movie 3 jewish', 'hitler was kil..."
4,"""In the world of old-school kung fu movies, wh...",positive,0.421275,"{'keyword': ['extraordinary kung fu', 'his lat..."
5,"Oh, dear! This has to be one of the worst film...",negative,0.420833,"{'keyword': ['the worst films', 'unbelievably ..."
6,This is a very memorable spaghetti western. It...,positive,0.415380,"{'keyword': ['good acting especially', 'evil v..."
7,"This was probably the worst movie ever, seriou...",negative,0.412661,"{'keyword': ['worst movie', 'watch it i', 'thi..."
8,I first saw this as a child living in East Lon...,positive,0.408642,"{'keyword': ['history in cinema', 'globally de..."
9,"Yes, I had the misfortune of watching this fil...",negative,0.395451,"{'keyword': ['other films ernest', 'inappropri..."


In [17]:
%%thanosql
SELECT * 
FROM (
    SELECT review, sentiment, json_array_elements(keyword -> 'keyword') AS keywords, (json_array_elements(keyword -> 'score'))::text::float AS score
    FROM (
        SEARCH KEYWORD
        USING movie_text_search_model
        OPTIONS (
            text_col="review",
            ngram_range=[1, 3],
            use_stopwords=True
            )
        AS (
            SELECT review, sentiment, score
            FROM (
                SEARCH TEXT 
                USING movie_text_search_model
                OPTIONS (
                    search_input_type="text",
                    search_input="The best action movie",
                    emb_col="convert_result",
                    column_name="score"
                    )
                AS 
                SELECT * 
                FROM movie_review_test
                WHERE review LIKE '%%gun%%'
                )
            ORDER BY score DESC 
            LIMIT 10
        )
    )
)
WHERE score > 0.3

,review,sentiment,keywords,score
0,As a veteran screen writing instructor at Rich...,positive,truly good movies,0.5779
1,As a veteran screen writing instructor at Rich...,positive,before hollywood started,0.4741
2,This is quite an unusual and unique little wes...,positive,western genre film,0.6365
3,This is quite an unusual and unique little wes...,positive,the movie seemed,0.5181
4,What was there about 1939 that helped produce ...,positive,melvyn douglas proved,0.3326
5,This is quite an unusual and unique little wes...,positive,james stewart as,0.3965
6,Countless Historical & cultural mistakes 0/10 ...,negative,stupid people,0.3139
7,"""In the world of old-school kung fu movies, wh...",positive,extraordinary kung fu,0.6047
8,This is quite an unusual and unique little wes...,positive,involved quite amazing,0.3180
9,What was there about 1939 that helped produce ...,positive,excellent hollywood films,0.6113


<div class="admonition note">
    <h4 class="admonition-title">Query Details</h4>
    <ul>
        <li>"<strong>SEARCH TEXT [image|audio|video|text|keyword]</strong>" defines the image|audio|video|text|keyword data type to search for.</li>
        <li>"<strong>SEARCH KEYWORD</strong>" uses the movie_text_search_model as an algorithm to search keywords.</li>
        <li>"<strong>USING</strong>" defines the model used for text vectorization.</li>
        <li>"<strong>AS</strong>" defines the embedding table to be used for searches. In this example, <code>movie_review_test</code> table is used.</li>
    </ul>
</div>

## __6. In Conclusion__

In this tutorial, we performed text vectorization using `movie review data`, and similar text search and keyword extraction. As this is a beginner-level tutorial, we focused on the process rather than accuracy. The model's accuracy can be improved by adjusting various options, such as increasing the epoch or dataset size. Create your own model and provide competitive services by combining various unstructured data (image, audio, video, etc.) and structured data with ThanoSQL.

* [How to Upload My Data to the ThanoSQL Workspace](https://docs.thanosql.ai/en/getting_started/data_upload/)
* [How to Create a Table Using My Data](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/COPY_SYNTAX/)
* [How to Upload My Model to the ThanoSQL Workspace](https://docs.thanosql.ai/en/how-to_guides/ThanoSQL_query/UPLOAD_SYNTAX/)

<div class="admonition tip">
    <h4 class="admonition-title">Inquiries about deploying a model for your own service</h4>
    <p>If you have any difficulties creating your own model using ThanoSQL or applying it to your services, please feel free to contact us below😊</p>
    <p>For inquiries regarding building an text similarity search model: contact@smartmind.team</p>
</div>